In [ ]:
#Imports
import subprocess
import os
import hydra
from flows.utils.general_helpers import read_yaml_file
from flows.backends.api_info import ApiInfo
from flows.utils.general_helpers import find_replace_in_dict
from flows.flow_launchers import FlowLauncher
from flows.backends.api_info import ApiInfo
from flows import flow_verse
from flows import logging



## Defining Functions Which Reproduce The Steps Of The Quick Start Tutorial

In [ ]:
DEPENDENCIES_DICT = {
    "ChatFlowModule": "aiflows/ChatFlowModule",
    "ReAct": "aiflows/ControllerExecutorFlowModule",
    "ChatInteractiveFlowModule": "aiflows/ChatInteractiveFlowModule",
    "ChatWithDemonstrationsFlowModule": "aiflows/ChatWithDemonstrationsFlowModule",
    "AutoGPTFlowModule": "aiflows/AutoGPTFlowModule",
    "VisionFlowModule": "aiflows/VisionFlowModule",
}

def download_all_flow_dependencies(flow):
    """Download all dependencies for a flow from FlowVerse and pip requirements. Note that
    thi doesn't work for all flows, only the ones that have been added to the DEPENDENCIES_DICT
    
    :param flow: The name of the flow to download dependencies for
    :type flow: str
    """
    # ~~~ Load Flow dependecies from FlowVerse ~~~
    dependencies = [
        {"url": DEPENDENCIES_DICT[flow], "revision": "main"},
    ]
    flow_verse.sync_dependencies(dependencies)
    
    # path to pip requirements file. Do not change this
    path_to_requirements = os.path.join("flow_modules" ,DEPENDENCIES_DICT[flow], "pip_requirements.txt")
    #download dependencies
    subprocess.check_call(["pip", "install", "-r", path_to_requirements], stdout=subprocess.DEVNULL)

def run_flow(flow, data, api_information):
    """ Installs the dependencies for a flow, loads the config, instantiates the flow and runs inference.
    Note that this doesn't work for all flows, only the ones that have been added to the DEPENDENCIES_DICT
    
    :param flow: The name of the flow to run
    :type flow: str
    :param data: The input data to the flow
    :type data: dict
    :param api_information: The api information for the flow
    :type api_information: ApiInfo
    """
    
    flow_url = DEPENDENCIES_DICT[flow]

    # ~~~ Load Flow dependecies from FlowVerse and pip requirments ~~~
    download_all_flow_dependencies(flow)

    # ~~~ Load Flow config ~~~
    path_to_config = os.path.join( "flow_modules" ,flow_url, "demo.yaml")
    cfg = read_yaml_file(path_to_config)

    #set api information
    cfg = find_replace_in_dict(cfg, key_to_find="api_infos", new_value= api_information , current_path="")
  
    # ~~~ Instantiate the Flow ~~~
    flow_with_interfaces = {
        "flow": hydra.utils.instantiate(cfg["flow"], _recursive_=False, _convert_="partial"),
        "input_interface": (
            None
            if cfg.get("input_interface", None) is None
            else hydra.utils.instantiate(cfg["input_interface"], _recursive_=False)
        ),
        "output_interface": (
            None
            if cfg.get("output_interface", None) is None
            else hydra.utils.instantiate(cfg["output_interface"], _recursive_=False)
        ),
    }
    
    # ~~~ Run inference ~~~
    path_to_output_file = None
    # path_to_output_file = "output.jsonl"  # Uncomment this line to save the output to disk

    _, outputs = FlowLauncher.launch(
        flow_with_interfaces=flow_with_interfaces, data=data, path_to_output_file=path_to_output_file
    )

    # ~~~ Print the output ~~~
    flow_output_data = outputs[0]
    print(flow_output_data)
    

## Setting you API Information

In [ ]:

# ~~~ Set the API information ~~~
# OpenAI backend
api_information = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]
# Azure backend
# api_information = ApiInfo(backend_used = "azure",
#                           api_base = os.getenv("AZURE_API_BASE"),
#                           api_key = os.getenv("AZURE_OPENAI_KEY"),
#                           api_version =  os.getenv("AZURE_API_VERSION") )

## Pick A Flow and Some Data and Run It!



In [ ]:
logging.set_verbosity_info()

# ~~~~ Pick a flow ~~~~
# Chose which flow you want to run
# Options: ChatFlowModule, ReAct, ChatInteractiveFlowModule,
#          ChatWithDemonstrationsFlowModule, AutoGPTFlowModule,
#          VisionFlowModule
flow = "ChatFlowModule"

# ~~~~ Get the data ! (uncomment the correct line) ~~~~
# Example data for ChatFlowModule
data = {"id": 0, "question": "What is the capital of France?"} 

# # Example data for ReAct (ControllerExecutorFlowModule)
#data = {"id": 0, "goal": "What is the capital of France?"}

# # Example data for ChatInteractiveFlowModule
# data = {"id": 0} # you will be able to ask questions once it is running

# # Example data for ChatWithDemonstrationsFlowModule
#data = {"id": 0, "question": "What's the capital of France?"}  # This can be a list of samples

# #example data for AutoGPTFlowModule
#data = {"id": 0, "goal": "Answer the following question: What is the profession and date of birth of Michael Jordan?"}

# #example data for VisionFlowModule
# url_image = {"type": "url",
#             "image": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"}

# data = {"id": 0, "query": "What’s in this image?", "data": {"images": [url_image]}}  # This can be a list of samples


run_flow(flow, data, api_information )
